### docker node update --availability pause

In [ ]:
# docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Active         Leader           20.10.13
y3ro2fe86icztsuasjqr5dgon     ip-172-31-34-88.ap-south-1.compute.internal    Ready     Active                          20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13

$ docker node update --availability pause ip-172-31-34-86.ap-south-1.compute.internal

ip-172-31-34-86.ap-south-1.compute.internal
[root@ip-172-31-34-86 ec2-user]# docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Pause          Leader           20.10.13
y3ro2fe86icztsuasjqr5dgon     ip-172-31-34-88.ap-south-1.compute.internal    Ready     Active                          20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13

### overlay 

In [ ]:
#Overlay used for communication of remote machines

In [ ]:
$ docker network create --driver overlay geolocation-net

4pfb1yq1fjd3up04tbdpvvsu4

$ docker network ls
NETWORK ID     NAME              DRIVER    SCOPE
9c40b87f6bf4   bridge            bridge    local
416d5d8e1641   docker_gwbridge   bridge    local
4pfb1yq1fjd3   geolocation-net   overlay   swarm
cda6d6a906a2   host              host      local
awgr73x9gv2r   ingress           overlay   swarm
7e1484f89127   none              null      local

## A manager, 4 api container relicas and 4 front end replicas in two worker instances and one redis container

### Labeling

In [ ]:
$ docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Pause          Leader           20.10.13
y3ro2fe86icztsuasjqr5dgon     ip-172-31-34-88.ap-south-1.compute.internal    Ready     Active                          20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13

#for redis container  label

$ docker node update --label-add resource=ram ip-172-31-37-234.ap-south-1.compute.internal

#for other two workers

$ docker node update --label-add resource=cpu ip-172-31-34-88.ap-south-1.compute.internal

$ docker node update --label-add resource=cpu ip-172-31-47-139.ap-south-1.compute.internal


In [ ]:
$ docker node inspect ip-172-31-37-234.ap-south-1.compute.internal|grep -i label -A 3

            "Labels": {
                "resource": "ram"
            },
            "Role": "worker",


### Creating redis container service

In [ ]:
$ docker service create \
--name geolocation-caching \
--constraint node.labels.resource==ram \
--replicas 1 \
--network geolocation-net \
redis:latest

In [ ]:
$ docker service ls

ID             NAME                  MODE         REPLICAS   IMAGE          PORTS
exjphcvbp7mw   geolocation-caching   replicated   1/1        redis:latest   
    
$ docker service ps geolocation-caching
ID             NAME                    IMAGE          NODE                                           DESIRED STATE   CURRENT STATE           ERROR     PORTS
jkxnan25ywnc   geolocation-caching.1   redis:latest   ip-172-31-37-234.ap-south-1.compute.internal   Running         Running 3 minutes ago 

### apicaching

In [ ]:
$ docker service create \
--name geolocation-api \
--constraint node.labels.resource==cpu \
--replicas 4 \
--network geolocation-net \
-e REDIS_PORT="6379" \
-e REDIS_HOST="geolocation-caching" \
-e APP_PORT="8080" \
-e API_KEY="6a77a7aac63e4ed69a78f5d6c8cbe0f4" \
jinumona/ipgeolocation-api-redis:v2

In [ ]:
$ docker service ls
ID             NAME                  MODE         REPLICAS   IMAGE                                 PORTS
g9z11k8gq20j   geolocation-api       replicated   4/4        jinumona/ipgeolocation-api-redis:v2   
exjphcvbp7mw   geolocation-caching   replicated   1/1        redis:latest 

$ docker service ps geolocation-api
ID             NAME                IMAGE                                 NODE                                           DESIRED STATE   CURRENT STATE           ERROR     PORTS
pclcstj07353   geolocation-api.1   jinumona/ipgeolocation-api-redis:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 2 minutes ago             
335j69lyqk02   geolocation-api.2   jinumona/ipgeolocation-api-redis:v2   ip-172-31-34-88.ap-south-1.compute.internal    Running         Running 2 minutes ago             
lodmal0mu9bu   geolocation-api.3   jinumona/ipgeolocation-api-redis:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 2 minutes ago             
u0z888vkk1tv   geolocation-api.4   jinumona/ipgeolocation-api-redis:v2   ip-172-31-34-88.ap-south-1.compute.internal    Running         Running 2 minutes ago 

### frontend

In [ ]:
$ docker service create \
--name geolocation-frontend \
--constraint node.labels.resource==cpu \
--replicas 4 \
--network geolocation-net \
-e API_SERVER="geolocation-api" \
-e API_SERVER_PORT="8080" \
-e APP_PORT="8080" \
-p 80:8080 \
jinumona/ipgeolocation-frontend:v2

In [ ]:
$ docker service ls
ID             NAME                   MODE         REPLICAS   IMAGE                                 PORTS
g9z11k8gq20j   geolocation-api        replicated   4/4        jinumona/ipgeolocation-api-redis:v2   
exjphcvbp7mw   geolocation-caching    replicated   1/1        redis:latest                          
vtlnkq3nm424   geolocation-frontend   replicated   4/4        jinumona/ipgeolocation-frontend:v2    *:80->8080/tcp


In [ ]:
$ docker service ps geolocation-frontend
ID             NAME                     IMAGE                                NODE                                           DESIRED STATE   CURRENT STATE                ERROR     PORTS
ywouifjv82lz   geolocation-frontend.1   jinumona/ipgeolocation-frontend:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running about a minute ago             
ewqm09zbaubz   geolocation-frontend.2   jinumona/ipgeolocation-frontend:v2   ip-172-31-34-88.ap-south-1.compute.internal    Running         Running about a minute ago             
ro0day6zrvzd   geolocation-frontend.3   jinumona/ipgeolocation-frontend:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running about a minute ago             
j848mfhmnvuk   geolocation-frontend.4   jinumona/ipgeolocation-frontend:v2   ip-172-31-34-88.ap-south-1.compute.internal    Running         Running about a minute ago             


#### we can access http://3.108.227.61/ip/8.8.8.8 from any server including manager,redis,workers etc

In [ ]:
manager >>[root@ip-172-31-34-86 ec2-user]# netstat -ntlp|grep 80
tcp6       0      0 :::80                   :::*                    LISTEN      3082/dockerd 
                        
redis>>[root@ip-172-31-37-234 ec2-user]# netstat -ntlp|grep 80
tcp6       0      0 :::80                   :::*                    LISTEN      3487/dockerd 

worker1>>[root@ip-172-31-47-139 ec2-user]# netstat -ntlp|grep 80
tcp6       0      0 :::80                   :::*                    LISTEN      3487/dockerd
                        
worker2>>[root@ip-172-31-34-88 ec2-user]# netstat -ntlp|grep 80
tcp6       0      0 :::80                   :::*                    LISTEN      3493/dockerd                        

### ALB 

In [ ]:
#target group 80. alb listener 80 and redorect to 443 and 443 lister with ACM certificate
add alb to route 53


In [ ]:
#  https://geolocation.inenso.in/ip/8.8.8.8
    
$ curl -IL http://geolocation.inenso.in
HTTP/1.1 301 Moved Permanently
Server: awselb/2.0
Date: Wed, 22 Jun 2022 07:08:44 GMT
Content-Type: text/html
Content-Length: 134
Connection: keep-alive
Location: https://geolocation.inenso.in:443/

HTTP/2 200 
date: Wed, 22 Jun 2022 07:08:45 GMT
content-type: text/html; charset=utf-8
content-length: 683
server: Werkzeug/2.0.2 Python/3.6.9

### Making one node for maintenance: drain mode

In [ ]:
$ docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Pause          Leader           20.10.13
y3ro2fe86icztsuasjqr5dgon     ip-172-31-34-88.ap-south-1.compute.internal    Ready     Active                          20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13

$ docker node update --availability drain ip-172-31-34-88.ap-south-1.compute.internal


$ docker service ps geolocation-frontend
ID             NAME                         IMAGE                                NODE                                           DESIRED STATE   CURRENT STATE                 ERROR     PORTS
ywouifjv82lz   geolocation-frontend.1       jinumona/ipgeolocation-frontend:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 38 minutes ago                  
4piainkv8rmp   geolocation-frontend.2       jinumona/ipgeolocation-frontend:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running about a minute ago              
ewqm09zbaubz    \_ geolocation-frontend.2   jinumona/ipgeolocation-frontend:v2   ip-172-31-34-88.ap-south-1.compute.internal    Shutdown        Shutdown about a minute ago             
ro0day6zrvzd   geolocation-frontend.3       jinumona/ipgeolocation-frontend:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 38 minutes ago                  
w2xzbmri8upw   geolocation-frontend.4       jinumona/ipgeolocation-frontend:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running about a minute ago              
j848mfhmnvuk    \_ geolocation-frontend.4   jinumona/ipgeolocation-frontend:v2   ip-172-31-34-88.ap-south-1.compute.internal    Shutdown        Shutdown about a minute ago             

$ docker service ls
ID             NAME                   MODE         REPLICAS   IMAGE                                 PORTS
g9z11k8gq20j   geolocation-api        replicated   4/4        jinumona/ipgeolocation-api-redis:v2   
exjphcvbp7mw   geolocation-caching    replicated   1/1        redis:latest                          
vtlnkq3nm424   geolocation-frontend   replicated   4/4        jinumona/ipgeolocation-frontend:v2    *:80->8080/tcp

$ docker service ps geolocation-api
ID             NAME                    IMAGE                                 NODE                                           DESIRED STATE   CURRENT STATE                 ERROR     PORTS
pclcstj07353   geolocation-api.1       jinumona/ipgeolocation-api-redis:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 58 minutes ago                  
9m7aaonk4fbq   geolocation-api.2       jinumona/ipgeolocation-api-redis:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running about a minute ago              
335j69lyqk02    \_ geolocation-api.2   jinumona/ipgeolocation-api-redis:v2   ip-172-31-34-88.ap-south-1.compute.internal    Shutdown        Shutdown about a minute ago             
lodmal0mu9bu   geolocation-api.3       jinumona/ipgeolocation-api-redis:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running 58 minutes ago                  
ged73fyit3hf   geolocation-api.4       jinumona/ipgeolocation-api-redis:v2   ip-172-31-47-139.ap-south-1.compute.internal   Running         Running about a minute ago              
u0z888vkk1tv    \_ geolocation-api.4   jinumona/ipgeolocation-api-redis:v2   ip-172-31-34-88.ap-south-1.compute.internal    Shutdown        Shutdown about a minute ago  

In [ ]:
$ docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Pause          Leader           20.10.13
y3ro2fe86icztsuasjqr5dgon     ip-172-31-34-88.ap-south-1.compute.internal    Ready     Drain                           20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13


### docker swarm leave

In [ ]:
#ssh  to swarm drained node

[root@ip-172-31-34-88 ec2-user]# docker swarm leave
Node left the swarm.


#### At manager

In [ ]:
$ docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Pause          Leader           20.10.13
y3ro2fe86icztsuasjqr5dgon     ip-172-31-34-88.ap-south-1.compute.internal    Down      Drain                           20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13



$ docker node rm ip-172-31-34-88.ap-south-1.compute.internal
ip-172-31-34-88.ap-south-1.compute.internal

$ docker node ls
ID                            HOSTNAME                                       STATUS    AVAILABILITY   MANAGER STATUS   ENGINE VERSION
ljcwwjcjqtivduc2hppuaalkq *   ip-172-31-34-86.ap-south-1.compute.internal    Ready     Pause          Leader           20.10.13
dupmpbug9qx4fgmzrdybbv5rd     ip-172-31-37-234.ap-south-1.compute.internal   Ready     Active                          20.10.13
we9w3fhr9djrt5p8p07hmyq5i     ip-172-31-47-139.ap-south-1.compute.internal   Ready     Active                          20.10.13